In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import geopandas as gpd
import seaborn as sns

#Packages
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
from scipy import stats
pd.set_option('display.max_columns', None)

from scipy.stats import skew, kurtosis

import pygris
import shapely

In [2]:
df = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/FimaNfipClaims.parquet.gzip")

## Blockgroup

In [3]:
chunk_size = 40000  # adjust based on your system's capabilities
chunks = [x for x in range(0, 320000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
BG_df = pd.concat(gdf_list, ignore_index=True)

In [6]:
BG_df.head()

,GEOID,geometry,year
0,020900009002,"POLYGON ((-147.96856 64.77841, -147.96852 64.7...",2020
1,020200007011,"POLYGON ((-149.77843 61.22629, -149.77740 61.2...",2020
2,021700007032,"POLYGON ((-149.63359 61.55695, -149.63313 61.5...",2020
3,021700007012,"POLYGON ((-149.72265 61.58833, -149.72221 61.5...",2020
4,021700013002,"POLYGON ((-149.17952 61.53109, -149.17766 61.5...",2020


In [4]:
# Read the shapefile
shapefile_path = 'C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/1980_bg/US_block_1980.shp' 
bg1980_df = gpd.read_file(shapefile_path)

In [28]:
bg1980_df[bg1980_df['GISJOIN'].str.len()== 15]

,GISJOIN,STATE,STATEA,SMSA,SMSAA,COUNTY,COUNTYA,TRACTA,BLOCKA,Shape_Area,geometry
0,G01007300001101,Alabama,01,"BIRMINGHAM, ALA.",1000,Jefferson,073,0001,101,9.576537e+03,"POLYGON ((854350.741 -397224.710, 854315.758 -..."
1,G01007300001102,Alabama,01,"BIRMINGHAM, ALA.",1000,Jefferson,073,0001,102,1.374698e+04,"POLYGON ((854274.699 -397299.497, 854279.056 -..."
2,G01007300001103,Alabama,01,"BIRMINGHAM, ALA.",1000,Jefferson,073,0001,103,3.882203e+04,"POLYGON ((854246.249 -397481.850, 854178.915 -..."
3,G01007300001105,Alabama,01,"BIRMINGHAM, ALA.",1000,Jefferson,073,0001,105,1.544605e+04,"POLYGON ((854179.051 -397174.193, 854196.477 -..."
4,G01007300001106,Alabama,01,"BIRMINGHAM, ALA.",1000,Jefferson,073,0001,106,1.390143e+04,"POLYGON ((854103.634 -397349.924, 854031.319 -..."
...,...,...,...,...,...,...,...,...,...,...,...
1020573,G55013302045631,Wisconsin,55,"MILWAUKEE, WIS.",5080,Waukesha,133,2045,631,3.226322e+06,"POLYGON ((609277.864 659873.307, 609282.431 65..."
1020574,G55013302045632,Wisconsin,55,"MILWAUKEE, WIS.",5080,Waukesha,133,2045,632,5.046098e+04,"POLYGON ((608850.875 660149.455, 608995.561 66..."
1020575,G55013302045633,Wisconsin,55,"MILWAUKEE, WIS.",5080,Waukesha,133,2045,633,3.607917e+04,"POLYGON ((609268.010 660062.524, 609134.995 66..."
1020576,G55013302045634,Wisconsin,55,"MILWAUKEE, WIS.",5080,Waukesha,133,2045,634,5.276360e+06,"POLYGON ((608949.000 659519.139, 608964.324 65..."


In [32]:
bg1980_df[(bg1980_df['GISJOIN'].str.len()== 17) & (bg1980_df['STATE'] == 'New York')]

,GISJOIN,STATE,STATEA,SMSA,SMSAA,COUNTY,COUNTYA,TRACTA,BLOCKA,Shape_Area,geometry
680087,G3600050002701101,New York,36,"NEW YORK, N.Y.-N.J.",5600,Bronx,005,002701,101,9277.226539,"POLYGON ((1831465.779 584062.865, 1831402.811 ..."
680088,G3600050002701102,New York,36,"NEW YORK, N.Y.-N.J.",5600,Bronx,005,002701,102,5154.040452,"POLYGON ((1831402.811 584001.928, 1831331.700 ..."
680089,G3600050002701201,New York,36,"NEW YORK, N.Y.-N.J.",5600,Bronx,005,002701,201,48494.260258,"POLYGON ((1831319.210 584028.176, 1831293.970 ..."
680090,G3600050002701301,New York,36,"NEW YORK, N.Y.-N.J.",5600,Bronx,005,002701,301,24598.086346,"POLYGON ((1831358.965 584187.192, 1831333.727 ..."
680091,G3600050002702101,New York,36,"NEW YORK, N.Y.-N.J.",5600,Bronx,005,002702,101,327.973027,"POLYGON ((1831764.320 584363.700, 1831745.052 ..."
...,...,...,...,...,...,...,...,...,...,...,...
730451,G3601190014904101,New York,36,"NEW YORK, N.Y.-N.J.",5600,Westchester,119,014904,101,314617.345068,"POLYGON ((1836180.686 645782.230, 1836152.392 ..."
730452,G3601190014904102,New York,36,"NEW YORK, N.Y.-N.J.",5600,Westchester,119,014904,102,148322.079742,"POLYGON ((1836676.394 646439.149, 1836655.598 ..."
730453,G3601190014904103,New York,36,"NEW YORK, N.Y.-N.J.",5600,Westchester,119,014904,103,318936.120963,"POLYGON ((1836121.239 645758.342, 1836152.392 ..."
730454,G3601190014904104,New York,36,"NEW YORK, N.Y.-N.J.",5600,Westchester,119,014904,104,115706.799248,"POLYGON ((1836040.751 645365.019, 1835860.118 ..."


In [24]:
bg1980_df['GISJOIN'].str.len().value_counts()

15    670495
17    350068
14        11
13         2
16         1
12         1
Name: GISJOIN, dtype: int64

In [16]:
bg1980_df[bg1980_df['BLOCKA'].isna()]

,GISJOIN,STATE,STATEA,SMSA,SMSAA,COUNTY,COUNTYA,TRACTA,BLOCKA,Shape_Area,geometry
388008,G2200710000612,Louisiana,22,"NEW ORLEANS, LA.",5560,Orleans,071,000612,NaN,4.566297e+04,"MULTIPOLYGON (((579924.349 -829529.538, 579970..."
389761,G2200710001730,Louisiana,22,"NEW ORLEANS, LA.",5560,Orleans,071,001730,NaN,1.203820e+05,"MULTIPOLYGON (((586696.357 -807062.153, 586677..."
389778,G2200710001731,Louisiana,22,"NEW ORLEANS, LA.",5560,Orleans,071,001731,NaN,2.153388e+06,"MULTIPOLYGON (((590209.106 -817641.999, 590186..."
389861,G2200710001732,Louisiana,22,"NEW ORLEANS, LA.",5560,Orleans,071,001732,NaN,2.112206e+05,"MULTIPOLYGON (((582176.881 -814954.643, 581984..."
392534,G22007100070,Louisiana,22,"NEW ORLEANS, LA.",5560,Orleans,071,0070,NaN,8.567526e+02,"POLYGON ((567789.241 -823426.006, 567832.460 -..."


In [9]:
bg1980_df.head()

,GISJOIN,STATE,STATEA,SMSA,SMSAA,COUNTY,COUNTYA,TRACTA,BLOCKA,Shape_Area,geometry
0,G01007300001101,Alabama,01,"BIRMINGHAM, ALA.",1000,Jefferson,073,0001,101,9576.537339,"POLYGON ((854350.741 -397224.710, 854315.758 -..."
1,G01007300001102,Alabama,01,"BIRMINGHAM, ALA.",1000,Jefferson,073,0001,102,13746.978680,"POLYGON ((854274.699 -397299.497, 854279.056 -..."
2,G01007300001103,Alabama,01,"BIRMINGHAM, ALA.",1000,Jefferson,073,0001,103,38822.033233,"POLYGON ((854246.249 -397481.850, 854178.915 -..."
3,G01007300001105,Alabama,01,"BIRMINGHAM, ALA.",1000,Jefferson,073,0001,105,15446.046669,"POLYGON ((854179.051 -397174.193, 854196.477 -..."
4,G01007300001106,Alabama,01,"BIRMINGHAM, ALA.",1000,Jefferson,073,0001,106,13901.428561,"POLYGON ((854103.634 -397349.924, 854031.319 -..."


In [ ]:
bg1980_df = bg1980_df.rename(columns={"BKGPIDFP00": "GEOID"})

bg1980_df = bg1980_df[['GEOID', 'geometry']]
bg1980_df['year'] = 1980

In [ ]:
list_censusBG = bg1980_df['GEOID'].dropna().drop_duplicates().astype(str).tolist()

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [ ]:
len(list_censusBG)

In [ ]:
#CensusBG list for NC

list_censusBG = df['censusBlockGroupFips'].dropna().drop_duplicates().tolist()

list_censusBG = [str(int(float(i))) for i in list_censusBG]

# add preceding 0 to make 4-digit zip codes into 5-digit
list_censusBG = [censusBG.zfill(12) for censusBG in list_censusBG]

In [ ]:
# Convert both lists to sets
set_bg = set(list_censusBG)
set_bg_2 = set(list_censusBG_2)

# Find the intersection of the two sets
common_bg = set_bg.intersection(set_bg_2)

# Print the number of common zip codes
print("Number of common BG:", len(common_bg))

In [ ]:
len(common_bg)/len(list_censusBG)

In [ ]:
len(common_bg)

In [ ]:
BG_list_df1980 = bg1980_df[bg1980_df['GEOID'].isin(common_bg)]

In [ ]:
BG_df_final = pd.concat([BG_df, BG_list_df1980], ignore_index=True)

In [ ]:
BG_df2 = BG_df_final.drop_duplicates()

In [ ]:
BG_df.shape[0] 

In [ ]:
BG_df_final.shape[0]

In [ ]:
BG_df2.shape[0]

In [ ]:
chunk_size = 30000  # adjust based on your system's capabilities
chunks = [x for x in range(0, len(BG_df), chunk_size)]

for start in chunks:
    end = start + chunk_size
    temp_df = BG_df.iloc[start:end].copy()
    temp_df['geometry'] = temp_df['geometry'].apply(lambda geom: geom.wkt)
    temp_df.to_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/BG_geometry_{start}_{end}.parquet.gzip", compression='gzip')

## Tract

In [ ]:
# Read the parquet file for Tract_shapefile
chunk_size = 30000 
chunks = [x for x in range(0, 180000, chunk_size)]

gdf_list = []

for start in chunks:
    end = start + chunk_size
    temp_df = pd.read_parquet(f"C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/Tract_geometry_{start}_{end}.parquet.gzip")
    gdf_read = gpd.GeoDataFrame(temp_df, geometry=temp_df['geometry'].apply(lambda x: shapely.wkt.loads(x)))
    gdf_list.append(gdf_read)

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
Tract_df= pd.concat(gdf_list, ignore_index=True)

In [33]:
# Read the shapefile
shapefile_path = 'C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/1980_tract/US_tract_1980.shp' 
tract1980_df = gpd.read_file(shapefile_path)

In [34]:
tract1980_df.head()

,NHGISST,NHGISCTY,GISJOIN,GISJOIN2,SHAPE_AREA,SHAPE_LEN,geometry
0,560,0210,G56002100016,56002100016,2.569850e+09,241261.569958,"POLYGON ((-732462.559 499607.011, -730487.717 ..."
1,560,0210,G56002100013,56002100013,9.690180e+06,14271.148305,"POLYGON ((-730612.550 448835.854, -730719.283 ..."
2,560,0210,G56002100014,56002100014,1.686311e+07,19833.251952,"POLYGON ((-728941.349 443959.363, -728943.472 ..."
3,560,0210,G56002100008,56002100008,4.748756e+06,10873.293718,"POLYGON ((-733509.072 444652.634, -733443.713 ..."
4,560,0210,G56002100009,56002100009,3.887032e+06,10440.687964,"POLYGON ((-734860.110 445048.418, -734804.170 ..."


In [37]:
tract1980_df['GISJOIN2'].str.len().value_counts()

11    30129
13    12788
Name: GISJOIN2, dtype: int64

In [39]:
tract1980_df[tract1980_df['GISJOIN2'].str.len()== 13]

,NHGISST,NHGISCTY,GISJOIN,GISJOIN2,SHAPE_AREA,SHAPE_LEN,geometry
12,550,0090,G5500090020501,5500090020501,2.620509e+07,24417.044171,"POLYGON ((625803.660 814434.821, 625597.010 81..."
13,550,0090,G5500090000303,5500090000303,1.846351e+06,5446.962948,"POLYGON ((629313.619 814035.161, 629228.110 81..."
32,550,0090,G5500090020701,5500090020701,3.692090e+07,31742.298260,"POLYGON ((637113.801 807017.410, 637184.442 80..."
41,550,0090,G5500090021302,5500090021302,1.218007e+07,20902.793649,"POLYGON ((627381.305 809531.901, 627456.408 80..."
47,550,0250,G5500250000502,5500250000502,8.350953e+06,13852.804927,"POLYGON ((527769.432 637284.174, 527864.826 63..."
...,...,...,...,...,...,...,...
42893,150,0030,G1500030008602,1500030008602,3.934827e+07,41128.847355,"POLYGON ((-6143751.505 293022.156, -6144302.37..."
42898,150,0030,G1500030010201,1500030010201,6.648920e+07,42209.900466,"MULTIPOLYGON (((-6117752.277 301270.081, -6117..."
42899,150,0030,G1500030008601,1500030008601,8.736330e+07,47659.031672,"POLYGON ((-6143009.804 295146.633, -6143080.16..."
42910,150,0090,G1500090030302,1500090030302,2.338019e+08,152723.219042,"MULTIPOLYGON (((-6043467.535 117844.229, -6043..."


## County

In [40]:
# Read the parquet file
df_read = pd.read_parquet("C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/County_geometry.parquet.gzip")

# Convert the WKT strings back to geometries
County_df = gpd.GeoDataFrame(df_read, geometry=df_read['geometry'].apply(lambda x: shapely.wkt.loads(x)))

County_df['year'] = County_df['year'].replace({2012: 2010, 2021: 2020})
County_df = County_df.drop_duplicates(subset=['CountyID', 'year'])

In [41]:
County_df.head()

,CountyID,geometry,year
0,31039,"POLYGON ((-96.55515 41.91587, -96.55515 41.914...",2020
1,53069,"POLYGON ((-123.49077 46.38358, -123.48813 46.3...",2020
2,31109,"POLYGON ((-96.68140 41.04566, -96.68139 41.045...",2020
3,31129,"POLYGON ((-98.04802 40.35066, -98.04674 40.350...",2020
4,72085,"POLYGON ((-65.85703 18.16468, -65.85733 18.164...",2020


In [35]:
# Read the shapefile
shapefile_path = 'C:/Users/Asus/Box/Flood Damage PredictionProject/Dataset/1980_county/US_county_1980.shp' 
county1980_df = gpd.read_file(shapefile_path)

In [36]:
county1980_df.head()

,DECADE,NHGISNAM,NHGISST,NHGISCTY,ICPSRST,ICPSRCTY,ICPSRNAM,STATENAM,ICPSRSTI,ICPSRCTYI,ICPSRFIP,STATE,COUNTY,PID,X_CENTROID,Y_CENTROID,GISJOIN,GISJOIN2,SHAPE_AREA,SHAPE_LEN,geometry
0,1980,Calhoun,280,0130,46,130,CALHOUN,Mississippi,0,0,0.0,280,0130,2234.0,6.110063e+05,-377494.736400,G2800130,2800130,1.522850e+09,166019.534275,"POLYGON ((618303.703 -361173.680, 618335.173 -..."
1,1980,Canadian,400,0170,53,170,CANADIAN,Oklahoma,0,0,0.0,400,0170,656.0,-1.781629e+05,-217409.002544,G4000170,4000170,2.344387e+09,210853.064017,"POLYGON ((-200984.587 -196445.623, -199458.771..."
2,1980,Nicholas,540,0670,56,670,NICHOLAS,West Virginia,0,0,0.0,540,0670,1877.0,1.311337e+06,193827.844299,G5400670,5400670,1.694900e+09,236544.667322,"POLYGON ((1308778.854 222439.958, 1308823.098 ..."
3,1980,Kankakee,170,0910,21,910,KANKAKEE,Illinois,0,0,0.0,170,0910,622.0,6.769275e+05,436630.571868,G1700910,1700910,1.764925e+09,186092.517925,"POLYGON ((704459.764 442224.287, 704486.550 44..."
4,1980,Dodge,270,0390,33,390,DODGE,Minnesota,0,0,0.0,270,0390,2096.0,2.506435e+05,734168.088876,G2700390,2700390,1.138611e+09,136328.026667,"POLYGON ((264605.980 753956.723, 264660.983 75..."
